# 01. Tabular Q Learning

Tabular Q Learning을 실습해봅니다.
- 모든 state의 value function을 table에 저장하고 테이블의 각 요소를 Q Learning으로 업데이트 하는 것으로 학습합니다.

## Colab 용 package 설치 코드

In [ ]:
!pip install gym

### package import

In [1]:
import tensorflow as tf
import numpy as np
import random
import gym
# from gym.wrappers import Monitor

np.random.seed(777)
tf.set_random_seed(777)

print("tensorflow version: ", tf.__version__)
print("gym version: ", gym.__version__)

tensorflow version:  1.8.0
gym version:  0.11.0


## Frozen Lake

**[state]**

        SFFF
        FHFH
        FFFH
        HFFG

    S : starting point, safe
    F : frozen surface, safe
    H : hole, fall to your doom
    G : goal, where the frisbee is located
    
**[action]**

    LEFT = 0
    DOWN = 1
    RIGHT = 2
    UP = 3

In [2]:
from IPython.display import clear_output

# Load Environment
env = gym.make("FrozenLake-v0")
# init envrionmnet
env.reset()
# only 'Right' action agent
for _ in range(5):
    env.render()
    next_state, reward, done, _ = env.step(2)


SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG


### Frozen Lake (not Slippery)

In [3]:
def register_frozen_lake_not_slippery(name):
    from gym.envs.registration import register
    register(
        id=name,
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4', 'is_slippery': False},
        max_episode_steps=100,
        reward_threshold=0.78, # optimum = .8196
    )

register_frozen_lake_not_slippery('FrozenLakeNotSlippery-v0')

In [4]:
env = gym.make("FrozenLakeNotSlippery-v0")
env.reset()
env.render()
'''
env.step()을 이용해서 Goal까지 직접 이동해보세요.
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3
'''
env.step(0); env.render()
# env.step(); env.render()


SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG


## Q-Learning
**Pseudo code**  
<img src="./img/qlearning_pseudo.png" width="80%" align="left">  

### Epsilon greedy

In [ ]:
# epsilon greedy policy

def epsilon_greedy_action(epsilon, n_action, state, q_table):
    
        # 구현해보세요.
        # if epsilon이 random 값보다 클때
            # random action
        # else
            # 가장 큰 Q값을 갖는 action을 고른다.
        
        return action

In [ ]:
# epsilon greedy test

epsilon = 0
q_table = np.array([[1,0,0,0],
                    [0,0,0,1],
                    [0,1,0,0]])
for state in range(3):
    action = epsilon_greedy_action(epsilon, 4, state, q_table)
    print("state: {}    action: {}".format(state, action))

### Q-value update

In [ ]:
def q_update(q_table, state, next_state, action, reward, alpha, gamma):
    
    # 구현해보세요.
    # update 수식은 pseudo code 참조
    # q_table[s, a] = q_table[s, a] + TD error
    
    return q_table

In [ ]:
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

q_table = np.array([[0,0,0,0],
                    [0,1,0,0]], dtype=np.float)
print("start\n", q_table)

reward = 1.0
alpha = 0.1
gamma = 0.9

for i in range(10):
    print("update {}".format(i))
    q_table = q_update(q_table, 0, 1, 2, reward, alpha, gamma)
    print(q_table)

### Agent class

## Goal에 도착하기 위해 생각해야 하는것
1. Goal에 한번이라도 도착해야만 reward가 나와서 update 된다 $\rightarrow$ goal에 어떻게 가게 할까?
2. hole에 빠졌을 때 episode가 끝나긴 하지만 reward에 차이는 없다. $\rightarrow$ hole에 빠져서 끝나면 negative reward를 주도록 한다.
3. 학습이 잘 되어도 epsilon 만큼의 확률로 random action을 한다. $\rightarrow$ 학습이 진행될수록 epsilon을 줄인다.

In [ ]:
class Tabular_Q_agent:
    def __init__(self, q_table, n_action, epsilon, alpha, gamma):
        self.q_table = q_table
        self.epsilon = epsilon
        self.alpha = alpha
        self.gamma = gamma
        self.n_action = n_action
    
    def get_action(self, state):
    
        # 구현해보세요. (e-greedy policy)
        
        return action
    
    def q_update(self, state, next_state, action, reward):
    
        # 구현해보세요.
        # update 수식은 pseudo code 참조
    
    return q_table

### Training agent

In [5]:
env = gym.make("FrozenLakeNotSlippery-v0")

EPISODE = 500
epsilon = 0.9
alpha = 0.8 # learning rate
gamma = 0.9 # discount factor
n_action = 

rlist = []
slist = []

is_render = False

# initialize Q-Table 
q_table = np.random.rand(env.observation_space.n, env.action_space.n)
print("Q table size: ", q_table.shape)

# agent 생성
agent = 

# Epiode 수만큼 반복
for e in range(EPISODE):
    state = env.reset()
    print("[Episode {}]".format(e))
    if is_render:
        env.render()
    
    total_reward = 0
    goal = 0
    done = False
    limit = 0
    
    # 게임이 끝날때까지 반복 또는 100번 step할 때까지 반복
    while not done and limit < 100:
        # 1. select action by e-greedy policy
        # e-greedy로 action을 선택.
            
        # 2. do action and go to next state
        # env.step()을 사용해 1 step 이동 후 next state와 reward, done 값을 받아옴.
        
        # 2.1. hole 에 빠졌을 때 (-) reward를 받도록 함.
        if reward == 1.0:
            print("GOAL")
            goal = 1
        # hole에 빠졌을 경우 -1 reward
        elif done:
            reward = reward - 1
        
        # 3. Q update
        # Q table에서 현재 state의 Q값을 update 한다.
        
        slist.append(state)
        state = next_state
        
        total_reward += reward
        limit += 1
        
    print(slist)
    slist = []
    print("total reward: ", total_reward)
    rlist.append(goal)
    
print("성공한 확률" + str(sum(rlist) / EPISODE) + "%")


SyntaxError: invalid syntax (<ipython-input-5-8f6fc8c8f91d>, line 16)

In [ ]:
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
print(agent.q_table)

### Test agent

In [ ]:
state = env.reset()
done = False
limit = 0

agent.epsilon = 0.0
while not done and limit < 30:
    action = agent.get_action(state)
    next_state, reward, done, _ = env.step(action)
    env.render()
    state = next_state
    limit += 1